Imports & Client Setup

In [3]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


Helper Function

In [4]:
def generate(prompt, temperature=0.5, max_tokens=200):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=1
    )
    return response.choices[0].message.content


SECTION 1 — ZERO-SHOT PROMPTING

## Zero-Shot Prompt

We provide *only* instructions, no examples.

### Review:
“The cupcakes tasted really good but the frosting was too sweet, and delivery arrived 30 minutes late.”


In [5]:
review_1 = """
The cupcakes tasted really good but the frosting was too sweet, 
and delivery arrived 30 minutes late.
"""

zero_shot_prompt = f"""
Summarize the following bakery review in 3 bullet points, focusing on taste and delivery:

Review: "{review_1}"
"""

zero_shot_output = generate(zero_shot_prompt)
zero_shot_output


'- The cupcakes had a delicious flavor.\n- The frosting was overly sweet for some tastes.\n- Delivery was delayed by 30 minutes.'

SECTION 2 — FEW-SHOT PROMPTING

## Few-Shot Prompt

We give the model multiple examples so it learns:
- tone  
- structure  
- summary format  


In [6]:
review_2 = """
The custom birthday cake looked beautiful, but the sponge was dry and lacked flavor. Not worth the price.
"""

few_shot_prompt = f"""
You are a bakery customer-experience analyst.
Summaries must always follow this format:

- Good aspects
- Problems
- Additional notes (if any)

Example 1:
Review: "The cupcakes tasted really good but the frosting was too sweet, and delivery arrived 30 minutes late."
Summary:
- Great cupcake flavor
- Frosting too sweet; delivery late
- Customer satisfied with taste overall

Example 2:
Review: "Absolutely loved the chocolate tart! Fresh, rich, and perfectly balanced."
Summary:
- Excellent freshness and rich flavor
- No issues reported
- Customer would order again

Now summarize this review using the same 3-line format:

Review: "{review_2}"
"""

few_shot_output = generate(few_shot_prompt)
few_shot_output


'- Beautiful appearance of the cake  \n- Dry sponge and lack of flavor; not worth the price  \n- Customer disappointed with taste and value  '

SECTION 3 — CHAIN-OF-THOUGHT PROMPTING

## Chain-of-Thought Prompt

We ask the model to think step-by-step before giving the final summary.

Review:
“The custom birthday cake looked beautiful, but the sponge was dry and lacked flavor. Not worth the price.”


In [7]:
cot_prompt = f"""
Analyze this bakery review step by step. Think through each stage carefully.

1. Identify positive elements.
2. Identify negative elements.
3. Identify emotional tone.
4. Give a final quality score from 1 to 5.
5. Provide a final short summary.

Review: "{review_2}"

Explain your reasoning step-by-step, then give the final answer.
"""

cot_output = generate(cot_prompt)
cot_output


'Let\'s analyze the bakery review step by step:\n\n1. **Identify positive elements**:\n   - The review mentions that "The custom birthday cake looked beautiful." This indicates a positive aspect regarding the visual appeal of the cake, which suggests that the presentation met the customer\'s expectations.\n\n2. **Identify negative elements**:\n   - The reviewer states that "the sponge was dry and lacked flavor." This indicates significant issues with the texture and taste of the cake, which are critical factors for any baked good.\n   - Additionally, the phrase "Not worth the price" suggests dissatisfaction with the overall value of the cake, indicating that the reviewer felt the quality did not justify the cost.\n\n3. **Identify emotional tone**:\n   - The emotional tone of the review is somewhat disappointed and dissatisfied. While the visual appeal is acknowledged positively, the issues with the cake\'s flavor and texture lead to a negative overall impression. The use of "not worth 

SECTION 4 — COMPARISON TABLE

In [8]:
df_compare = pd.DataFrame({
    "Prompt Type": ["Zero-Shot", "Few-Shot", "Chain-of-Thought"],
    "Output": [zero_shot_output, few_shot_output, cot_output]
})

df_compare


,Prompt Type,Output
0,Zero-Shot,- The cupcakes had a delicious flavor.\n- The ...
1,Few-Shot,- Beautiful appearance of the cake \n- Dry sp...
2,Chain-of-Thought,Let's analyze the bakery review step by step:\...


# Observations

- Zero-shot produced a general summary but lacked consistency.
- Few-shot produced structured, predictable outputs matching the examples.
- Chain-of-thought gave the deepest analysis with step-by-step reasoning.
- Few-shot is best for consistent summaries.
- CoT is best for insight and customer sentiment analysis.
